In [1]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("tf_idf").getOrCreate()

In [2]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [3]:
sentenceData = spark.createDataFrame([(0.0, "Hi I heard about Spark"),
                               (1.0, "I wish java could use case classes"),
                               (2.0, "Logistic regression models are neat")],
                              ["label", "sentence"])

In [4]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  1.0|I wish java could...|
|  2.0|Logistic regressi...|
+-----+--------------------+



In [5]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [6]:
words_data = tokenizer.transform(sentenceData)

In [7]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  1.0|I wish java could...|[i, wish, java, c...|
|  2.0|Logistic regressi...|[logistic, regres...|
+-----+--------------------+--------------------+



In [8]:
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures")

In [9]:
featurized_data = hashing_tf.transform(words_data)

In [10]:
featurized_data.show()

+-----+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[24417,49...|
|  1.0|I wish java could...|[i, wish, java, c...|(262144,[20719,24...|
|  2.0|Logistic regressi...|[logistic, regres...|(262144,[13671,91...|
+-----+--------------------+--------------------+--------------------+



In [11]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [12]:
idf_model = idf.fit(featurized_data)

In [13]:
rescaled_data = idf_model.transform(featurized_data)

In [14]:
rescaled_data.show()

+-----+--------------------+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[24417,49...|(262144,[24417,49...|
|  1.0|I wish java could...|[i, wish, java, c...|(262144,[20719,24...|(262144,[20719,24...|
|  2.0|Logistic regressi...|[logistic, regres...|(262144,[13671,91...|(262144,[13671,91...|
+-----+--------------------+--------------------+--------------------+--------------------+



In [15]:
from pyspark.ml.feature import CountVectorizer

In [17]:
df = spark.createDataFrame([(0, "a b c".split()),
                           (1, "a b b c a".split())],
                          ["id", "words"])

In [18]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [21]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [22]:
model = cv.fit(df)

In [23]:
result = model.transform(df)

In [25]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

